#algorithm

##Library

In [ ]:
!pip install geopy
!pip install geopy timezonefinder
!pip install xlsxwriter
!pip install "git+https://github.com/oemof/feedinlib.git" -q
!pip install siphon
!pip install pvlib
!pip install NREL-PySAM
!pip install pyspark
import os
import requests
import xarray as xr
import csv
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pvlib
from pvlib import pvsystem
from pvlib.forecast import GFS, NAM, NDFD, HRRR, RAP
from feedinlib import WindPowerPlant
from feedinlib import get_power_plant_data
from feedinlib import era5
from geopy.geocoders import Nominatim
from timezonefinder import TimezoneFinder
from datetime import datetime, timedelta
from matplotlib.dates import DateFormatter
from sklearn.preprocessing import MinMaxScaler
from pyspark.sql.functions import col, pandas_udf
from pyspark.sql.types import LongType
from requests import get

In [ ]:
uid = '170535'
keypart = '8eda45d9-e9a2-468d-8154-732dd3a40435'
with open('/root/.cdsapirc','w') as keyFile:
      keyFile.write(
          'url: https://cds.climate.copernicus.eu/api/v2\n' + 
          f'key: {uid}:{keypart}'
    )
!cat '/root/.cdsapirc'

openenergy = get('https://openenergy-platform.org/api/v0/schema/supply/tables/wind_turbine_library/rows')

##Class

In [ ]:
class multy:
  def __init__(self, city, start, end):
    geolocator = Nominatim(user_agent="user")
    location = geolocator.geocode(city)
    tf = TimezoneFinder()
    loc = (location.latitude, location.longitude)
    timezone = tf.timezone_at(lng=location.longitude, lat=location.latitude)
    locations = [(location.latitude, location.longitude)]
    self.start = start
    self.end = end
    self.latitude = location.latitude
    self.longitude = location.longitude
    self.altitude = location.altitude

    base_url_csv = "https://power.larc.nasa.gov//api/temporal/hourly/point?parameters=WS10M,ALLSKY_SFC_SW_DIFF,WS50M,T2M,PS,ALLSKY_SFC_SW_DWN,CLRSKY_SFC_SW_DWN,RH2M&community=SB&longitude={longitude}&latitude={latitude}&start={start}&end={end}&format=CSV"
    filename_template = "Weather.csv"
    for latitude, longitude in locations:
        api_request_url = base_url_csv.format(longitude=longitude, latitude=latitude, start=start, end=end)
        response = requests.get(url=api_request_url, verify=True, timeout=30.00)
        filename = filename_template.format(latitude=latitude, longitude=longitude)

    with open(os.path.split(filename)[1], 'wb') as f, \
            requests.get(api_request_url, stream=True) as r:
       for line in r.iter_lines():
            f.write(line+'\n'.encode())

  def weather(self):
    custom_datetime = lambda x: datetime.strptime(x, '%d %m %Y %H')
    df = pd.read_csv('/content/Weather.csv', skiprows=16, encoding='utf-8', parse_dates={ 'Date_Time': ['DY', 'MO', 'YEAR', 'HR'] }, date_parser=custom_datetime
                      )
    df.rename(columns = {'T2M':'Tamb','PS':'pressure', 'Date_Time': 'time','ALLSKY_SFC_SW_DWN': 'GHI', 'CLRSKY_SFC_SW_DWN': 'DHI', 'RH2M':'RH', 'ALLSKY_SFC_SW_DIFF': 'dni'}, inplace = True)
    df['temperature'] = df['Tamb'] + 273.15 
    df['pressure'] = df['pressure']*1000
    df.index.rename('time')
    return df

  def windpowerlib_weather(self):

    date1  = datetime.strptime(str(self.start), "%Y%m%d")
    date2 = date1 + timedelta(days=1)
    start_date, end_date = "{}-{}-{}".format(date1.year, date1.month, date1.day), "{}-{}-{}".format(date2.year, date2.month, date2.day)
    # установить переменную, установленную для загрузки
    variable = "windpowerlib"
    target_file = 'ERA5_pvlib.nc'
  
    ds = era5.get_era5_data_from_datespan_and_position(
      variable=variable,
      start_date=start_date, end_date=end_date, 
      latitude=self.latitude, longitude=self.longitude,
      target_file=target_file)
  
    era5_netcdf_filename = 'ERA5_pvlib.nc'
    ds = xr.open_dataset(era5_netcdf_filename)
    ds2 = era5.format_windpowerlib(ds)

    custom_datetime = lambda x: datetime.strptime(x, '%d %m %Y %H')
    df = pd.read_csv('/content/Weather.csv', skiprows=16, encoding='utf-8', parse_dates={ 'Date_Time': ['DY', 'MO', 'YEAR', 'HR'] }, date_parser=custom_datetime
                      )
    df.rename(columns = {'T2M':'temperature','PS':'pressure', 'Date_Time': 'time'}, inplace = True)
    df.drop(['ALLSKY_SFC_SW_DWN','CLRSKY_SFC_SW_DWN','ALLSKY_SFC_SW_DIFF','RH2M'], axis = 1 , inplace = True)
    df['temperature'] = df['temperature'] + 273.15 
    df['pressure'] = df['pressure']*1000
    df['roughness_length'] = round(ds2.iloc[1]['roughness_length'].item(),2)
    df['latitude'] = round(self.latitude,1)
    df['longitude'] = round(self.longitude,1)
    df = df.set_index(['time', 'latitude', 'longitude'])
    midx = pd.MultiIndex(
    levels=[
        ["wind_speed","pressure","temperature","roughness_length"],
        [0,2,10,50]
    ],
    codes=[
        [0,0,1,2,3],
        [2,3,0,1,0]
    ],
    names=["variable","height"],
    )
    df.columns = midx
    df.dropna(inplace=True)
    df
    return df

  def PVlib_weather(self):
    custom_datetime = lambda x: datetime.strptime(x, '%d %m %Y %H')
    df = pd.read_csv('/content/Weather.csv', skiprows=16, encoding='utf-8', parse_dates={ 'Date_Time': ['DY', 'MO', 'YEAR', 'HR'] }, date_parser=custom_datetime
                      )
    df.rename(columns = {'T2M':'Tamb','PS':'pressure', 'Date_Time': 'time','ALLSKY_SFC_SW_DWN': 'GHI', 'CLRSKY_SFC_SW_DWN': 'DHI', 'RH2M':'RH', 'SZA' : 'zenith', 'ALLSKY_SFC_SW_DIFF': 'dni'}, inplace = True)
    df['temperature'] = df['Tamb'] + 273.15 
    df['pressure'] = df['pressure']*1000
    df.set_index(['time'], inplace= True)
    df.index.name = None
    df.drop(['temperature','WS50M','pressure','RH'], axis = 1 , inplace = True)
    df.rename(columns = {'WS10M': 'wind_speed', 'GHI': 'ghi', 'DHI': 'dhi', 'Tamb': 'temp_air'}, inplace = True)
    df = df.reindex(columns=['ghi', 'dhi', 'dni', 'temp_air', 'wind_speed'])
    df
    return df

In [ ]:
def Wind(Wind_power, windpowerlib_weather:pd.Series) -> pd.Series:
  df = pd.DataFrame(openenergy.json())
  df = df[['manufacturer', 'name','turbine_type', 'nominal_power', 'rotor_diameter', 'rotor_area', 'hub_height', 'has_power_curve', 'has_cp_curve']]
  df["has_power_curve"] = df["has_power_curve"].astype(int)
  df["has_cp_curve"] = df["has_cp_curve"].astype(int)
  df = df.loc[((df['has_power_curve'] == 1) &(df['has_cp_curve' ] == 1))]
  d = min(sorted(df['nominal_power']), key=lambda x: abs(Wind_power - x))
  df = df.loc[((df['nominal_power'] <= d)&((df['rotor_diameter'].astype(int)/2)<= 50))]
  df.reset_index(drop=True, inplace=True)
  turbine_data = {
    'turbine_type': df.at [0, 'turbine_type'],  #название турбины, как в библиотеке турбин
    'hub_height': 50  # in m
    }
  wind_turbine = WindPowerPlant(**turbine_data)
  feedin = wind_turbine.feedin(
    weather=windpowerlib_weather)
  result_wind = pd.DataFrame(feedin)
  result_wind = result_wind.reset_index( level = [1 , 2] )
  return result_wind

In [ ]:
tom = multy('Kemerovo', 20191231, 20210201)
